In [3]:
import json
import numpy as np
import pandas as pd
from collections import defaultdict
import pickle
import plot_function
import map_visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.ticker as ticker
import sys
from resizeimage import resizeimage
import os
from PIL import Image, ImageOps

In [4]:
#provincia_regione = pd.read_table("Data_final/regioni.csv", sep = "\t")
regione_zona = pd.read_table("Data_final/territori.csv", sep = "\t")
# to have a more readible db
regione_zona["Regione"] = regione_zona["Regione"].replace({"Provincia Autonoma Bolzano / Bozen": "Provincia Autonoma di Bolzano", 
                                                                    "Provincia Autonoma Trento": "Provincia Autonoma di Trento", 
                                                                    "Valle d'Aosta / Vallée d'Aoste": "Valle d'Aosta"})
#palette_continent = json.load(open('Statistics/color_continent_dict.json'))
zone_countries = json.load(open('Statistics/py_zone_country.json'))
zone_countries = {k1: {k2:[int(i) for i in v2] for k2, v2 in v1.items()} for k1, v1 in zone_countries.items()}
top5_region = json.load(open('Statistics/py_region_top5_countrie_cont.json'))
top5_region = {k1: {k2:[int(i) for i in v2] for k2, v2 in v1.items()} for k1, v1 in top5_region.items()}

continent = json.load(open('Statistics/continent_dict.json'))

all_cont = sorted(continent.keys())
all_r = sorted(set(regione_zona["Regione"]))
all_z = sorted(set(regione_zona["Zona"]))

In [5]:
def topn_zone(zones, r_z, top5_r, cont_dict, cont):
    temp = defaultdict(list)
    res_r = {z: {} for z in zones}
    for z in zones:
        z_regions = list(r_z[r_z["Zona"] == z]["Regione"].values)
        for r in z_regions:
            c_list = [c for c in top5_r[r].keys() if c in cont_dict[cont]]
            temp[z] += c_list
            res_r[z].update({r: {c: top5_region[r][c] for c in c_list}})
        temp[z] = list(set(temp[z]))
    res_z = {r: {c: zone_countries[r][c] for c in temp[r]} for r in temp.keys()}
    return(res_z, res_r)

In [6]:
'''topn_z_Africa, topn_r_Africa = topn_zone(all_z, regione_zona, top5_region, continent, "Africa")
topn_z_America, topn_r_America = topn_zone(all_z, regione_zona, top5_region, continent, "America")
topn_z_Asia, topn_r_Asia = topn_zone(all_z, regione_zona, top5_region, continent, "Asia")
topn_z_Europe, topn_r_Europe = topn_zone(all_z, regione_zona, top5_region, continent, "Europe")
topn_z_Pacific, topn_r_Pacific = topn_zone(all_z, regione_zona, top5_region, continent, "Pacific")'''

'topn_z_Africa, topn_r_Africa = topn_zone(all_z, regione_zona, top5_region, continent, "Africa")\ntopn_z_America, topn_r_America = topn_zone(all_z, regione_zona, top5_region, continent, "America")\ntopn_z_Asia, topn_r_Asia = topn_zone(all_z, regione_zona, top5_region, continent, "Asia")\ntopn_z_Europe, topn_r_Europe = topn_zone(all_z, regione_zona, top5_region, continent, "Europe")\ntopn_z_Pacific, topn_r_Pacific = topn_zone(all_z, regione_zona, top5_region, continent, "Pacific")'

In [57]:
def topn_zone_region_plot(all_z, regione_zona, top5_region, continent, continent_name):
    # list of top countries for a specific Continent-zone
    # list of top countries for a specific Continent-Region (only for the regions of a specific zone)
    topn_z_, topn_r_ = topn_zone(all_z, regione_zona, top5_region, continent, continent_name)
    # define the palette - assigne a color to each country in the top_zone list
    colors_list = ['darkgreen', 'yellowgreen', 'gold', 'darkorange', 'red', 'brown', 'paleturquoise', 
     'deepskyblue', 'darkblue', 'mediumpurple', 'hotpink', 'pink']
    
    # define a unique palette dict
    all_c = set()
    for zone in all_z:
        # sorted list of n top countries for the zone - merge of top 5 countries for each region in the zone
        all_c.update(list(topn_z_[zone].keys()))
    all_c = sorted(list(all_c))
    colors_palette = {c: col for c, col in zip(all_c, colors_list)}
    
    years = list(range(2003, 2018))
    for zone in all_z:
        # sorted list of n top countries for the zone - merge of top 5 countries for each region in the zone
        top_zone = sorted(list(topn_z_[zone].keys()))

    years = list(range(2003, 2018))
    for zone in all_z:
        # sorted list of n top countries for the zone - merge of top 5 countries for each region in the zone
        top_zone = sorted(list(topn_z_[zone].keys()))
        # top 5 countries for each region in the zone
        top_region = topn_r_[zone]

        # define a single dataframe - useful for the plot
        temp = {}
        for r in list(top_region.keys()):
            temp[r] = pd.DataFrame.from_dict(top_region[r])
            temp[r]["Region"] = [r for i in temp[r].index]

        temp_concat = pd.concat([temp[r] for r in list(top_region.keys())])
        
        temp_top = topn_z_[zone]
        
        plot_function.line_per_year(temp_top, years, zone, "Abs Population", 
                                    continent_name+" top n countries distribution in "+zone, colors_palette, 
                                    top_zone, save = True, path = "Temp_plot/"+continent_name+"_"+zone, 
                                    preprocess=True, sep_legend = True)
        # col_wrap: quanti grafici per ogni riga
        sns.set_style("whitegrid")
        sns.set_context({"figure.figsize": (10, 8)})
        g = sns.FacetGrid(temp_concat, col="Region", col_wrap=2, size=5)
        for c in top_zone:
            g = g.map(plt.plot, c, marker=".", color = colors_palette[c])
        # iterate over axes of FacetGrid
        for ax in g.axes.flat:
            labels = list(range(2001, 2018, 2))
            ax.locator_params(nbins=8)
            ax.set_xticklabels(labels, rotation=30) 
        g.set_xlabels("Years") 
        g.set_ylabels("Value") 
        #plt.show()
        g.fig.subplots_adjust(top=0.9)
        g.fig.suptitle("Top 5 countries distribution in the "+zone+" regions", fontsize = 16)
        #plt.show()
        plt.savefig("Temp_plot/regions_"+continent_name+"_"+zone+".png", bbox_inches='tight')
        plt.close()
        result = Image.new("RGB", (1248, 547))

        img = Image.open("Temp_plot/"+continent_name+"_"+zone+".png")
        img.thumbnail((624, 512), Image.ANTIALIAS)
        x = 0 * 624
        y = 0
        w, h = img.size
        #print('pos {0},{1} size {2},{3}'.format(x, y, w, h))
        img = ImageOps.expand(img, border=(int((1248-624-h)/2), 0, 547, 1248), fill='white')
        w, h = img.size
        #print('pos {0},{1} size {2},{3}'.format(x, y, w, h))
        result.paste(img, (x, y, x + w, y + h))

        img = Image.open("Temp_plot/regions_"+continent_name+"_"+zone+".png")
        img.thumbnail((624, 512), Image.ANTIALIAS)
        x = 1 * 624
        y = 0
        w, h = img.size
        #print('pos {0},{1} size {2},{3}'.format(x, y, w, h))
        if zone != "Isole":
            img = ImageOps.expand(img, border=(int((1248-624-h)/2), 0, 547, 1248), fill='white')
        else:
            img = ImageOps.expand(img, border=(0, 0, 547, 1248), fill='white')
        w, h = img.size
        #print('pos {0},{1} size {2},{3}'.format(x, y, w, h))
        result.paste(img, (x, y, x + w, y + h))

        img = Image.open("Temp_plot/legend_"+continent_name+"_"+zone+".png")
        w, h = img.size
        border_x = int((1248 - w)/2)
        img = ImageOps.expand(Image.open("Temp_plot/legend_"+continent_name+"_"+zone+".png"), border=(border_x, 0), fill='white')
        w, h = img.size
        #print(w, h)
        result.paste(img, (0, 512, 0 + w, 512 + h))

        result.save(os.path.expanduser("Plots/complete"+continent_name+"_"+zone+".png"))

In [58]:
for continent_name in list(continent.keys()):
    topn_zone_region_plot(all_z, regione_zona, top5_region, continent, continent_name)